In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from fractions import Fraction as frac
from decimal import Decimal
import regex as re

In [1]:
url = 'https://maplestory.nexon.com/Guide/OtherProbability/cube/red'


In [2]:
def process_df(df,first=True,firstweights=None,primeprob=None):
    split = df.iloc[:,0].str.split(':',expand=True)
    split.columns = ['Stat','Value']
    split.Stat.loc[split.Value.str.contains('%',na=False) & (split.Stat != '모든  스킬의 재사용 대기시간 ')] = split.Stat.loc[split.Value.str.contains('%',na=False)] + "%"
    split.Value = split.Value.str.extract('([0-9]+)').astype('int',errors='ignore')
    split.fillna({'Value':1},inplace=True)
    split.Stat = split.Stat.str.strip()
    probs = df.iloc[:,1].apply(lambda x: float(x.strip('%'))/100)
    if first:
        reduced = probs/min(probs)
        weights = reduced.apply(lambda x: frac(Decimal(x)).limit_denominator(100) * max(reduced.apply(lambda x: frac(Decimal(x)).limit_denominator(100).denominator))).astype('int')
    else:
        weights = pd.Series(index=probs.index)
        weights[-len(firstweights):] = firstweights
        weights[:-len(firstweights)] = probs[:-len(firstweights)] / primeprob * sum(firstweights)
        weights = weights.apply(lambda x: frac(Decimal(x)).limit_denominator(20) * max(weights.apply(lambda x: frac(Decimal(x)).limit_denominator(20).denominator)))
    split['Weight'] = weights
    return split



In [3]:
def get_cube_weights(url):
    html = requests.get(url).content
    df_list = pd.read_html(html, match='STR')
    soup = BeautifulSoup(html, 'html.parser')
    table_properties = soup.find_all('div',{'class': 'cube_option'})
    cube_df = []
    for i in range(0,len(df_list)):
        metadata = re.findall('<span>(.+)</span>',str(table_properties[i]))
        if metadata[0] == '레어':
            no_rare_lines = len(df_list[i].iloc[:,0:2].dropna())
            normal_lines = df_list[i].iloc[:-no_rare_lines,2:4].dropna()
            minidf = process_df(normal_lines)
            minidf['Grade'] = 'Normal'
            minidf['Equip'] = metadata[1]
            minidf['Level Requirement'] = int(re.findall('([0-9]+)',metadata[2])[0])
            minidf = minidf[['Equip','Level Requirement','Grade','Stat','Value','Weight']]
            cube_df.append(minidf)
        primelines = df_list[i].iloc[:,0:2].dropna()
        minidf = process_df(primelines)
        minidf['Grade'] = metadata[0]
        minidf['Equip'] = metadata[1]
        minidf['Level Requirement'] = int(re.findall('([0-9]+)',metadata[2])[0])
        minidf = minidf[['Equip','Level Requirement','Grade','Stat','Value','Weight']]
        cube_df.append(minidf)

    alldf = pd.concat(cube_df,ignore_index=True)
    return alldf

In [6]:
cubedict = {'Red':'https://maplestory.nexon.com/Guide/OtherProbability/cube/red',
           'Black': 'https://maplestory.nexon.com/Guide/OtherProbability/cube/black',
           'Additional': 'https://maplestory.nexon.com/Guide/OtherProbability/cube/addi'}
df = []
for cube,url in cubedict.items():
    cubeweights = get_cube_weights(url)
    cubeweights['Cube'] = cube
    df.append(cubeweights)

masterlist = pd.concat(df,ignore_index=True)
masterlist.to_csv('weights_kr.csv',index=False)

In [8]:
# koreancolumns = masterlist[['Equip','Grade','Stat']]
# koreanwords = []
# for i in koreancolumns.columns:
#     unique = koreancolumns[i].unique()
#     koreanwords += [x for x in unique]
# with open('koreanwords.csv','w+') as f:
#     for j in koreanwords:
#         f.write('"'+j+'"\n')

dictionary = pd.read_excel('translations.xlsx')
def translate(df,fromlang='kr',tolang='en'):
    translation_map = dictionary.set_index(fromlang)[tolang].to_dict()
    df2 = df.copy()
    for i in df.columns:
        df2[i] = df[i].map(lambda x: translation_map.get(x,x))
    return df2

translate(masterlist).to_csv('weights_en.csv',index=False)

In [7]:
masterlist['Stat'].unique()

array(['STR', 'DEX', 'INT', 'LUK', '최대 HP', '최대 MP', '공격력', '마력', 'STR %',
       'DEX %', 'INT %', 'LUK %', '공격력 %', '마력 %', '크리티컬 확률 %', '데미지 %',
       '올스탯', '공격 시 20% 확률로 240의 HP 회복', '공격 시 20% 확률로 120의 MP 회복',
       '공격 시 20% 확률로 6레벨 중독효과 적용', '공격 시 10% 확률로 2레벨 기절효과 적용',
       '공격 시 20% 확률로 2레벨 슬로우효과 적용', '공격 시 20% 확률로 3레벨 암흑효과 적용',
       '공격 시 10% 확률로 2레벨 빙결효과 적용', '공격 시 10% 확률로 2레벨 봉인효과 적용',
       '몬스터 방어율 무시 %', '최대 HP %', '최대 MP %', '올스탯 %',
       '공격 시 20% 확률로 360의 HP 회복', '공격 시 20% 확률로 180의 MP 회복',
       '보스 몬스터 공격 시 데미지 %', '캐릭터 기준 10레벨 당 공격력', '캐릭터 기준 10레벨 당 마력',
       '공격 시 20% 확률로 200의 HP 회복', '공격 시 20% 확률로 110의 MP 회복',
       '공격 시 20% 확률로 5레벨 중독효과 적용', '공격 시 20% 확률로 2레벨 암흑효과 적용',
       '공격 시 20% 확률로 300의 HP 회복', '공격 시 20% 확률로 165의 MP 회복',
       '피격 시 5% 확률로 데미지의 20% 무시', '피격 시 5% 확률로 데미지의 40% 무시',
       '피격 시 10% 확률로 데미지의 20% 무시', '피격 시 10% 확률로 데미지의 40% 무시', '방어력',
       '방어력 %', '모든  스킬레벨', 'HP 회복 아이템 및 회복 스킬 효율 %',
       '<쓸만한 미스틱 도어> 스킬 사용  가능', '모든  스킬